In [1]:
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import keras
import resnet
import keras.backend as K

from skimage import transform
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from MyNetWork import NetBuilder


Using TensorFlow backend.


In [2]:
num_classes = 2

# input image dimensions
img_rows, img_cols = 128, 128
input_shape = (img_rows, img_cols, 1)

Input = np.load('Data/final_data.npz')

data = Input["data"]
label = Input["label"]

print (data.shape)
print (label.shape)

(1022, 128, 128, 1)
(1022, 2)


In [3]:
def unison_shuffled_copies(a, b,rl):
    assert len(a) == len(b)
    r=range(len(a))
#     random.shuffle(r,lambda:rl)
    random.shuffle(r,lambda:rl)
    p=np.array(r)
    #p = np.random.permutation(len(a))
    return a[p], b[p]

data,label=unison_shuffled_copies(data,label,0.5)

splitpoint = int(round(len(data) * 0.8))
(x_train_arr, x_test_arr) = (data[0:splitpoint], data[splitpoint:])
(y_train_arr, y_test_arr) = (label[0:splitpoint], label[splitpoint:])

In [4]:
print('x_train shape:', x_train_arr.shape)
print('y_train_shape:', y_train_arr.shape)
print(x_train_arr.shape[0], 'train samples')
print(x_test_arr.shape[0], 'test samples')

num_0=0
for i in range(len(x_test_arr)):
    if y_test_arr[i][1]==1:
        num_0+=1
print(num_0)

('x_train shape:', (818, 128, 128, 1))
('y_train_shape:', (818, 2))
(818, 'train samples')
(204, 'test samples')
113


In [5]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
#lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('Test.csv')

batch_size = 64
nb_classes = 2
nb_epoch = 200
data_augmentation = True
Mysgd = keras.optimizers.SGD(lr=0.01)
MyAdam = keras.optimizers.Adam(lr=0.1)

#model = NetBuilder.Res50ST(input_shape = (128, 128, 1), num_outputs = nb_classes, st_pos=0, sampling_size = (32, 32))
#model = NetBuilder.ST_res50(input_shape = (128, 128, 1), num_outputs = nb_classes, sampling_size = (128, 128))
model = resnet.ResnetBuilder.build_resnet_50((1, 128, 128), nb_classes)
#model = NetBuilder.build_STres50ST(input_shape = (96, 96, 1), num_outputs = nb_classes, st_pos=-1, sampling_size=(30, 30))
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [6]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train_arr, y_train_arr,
             batch_size = batch_size,
             nb_epoch = nb_epoch,
             validation_data = (x_test_arr, y_test_arr),
             shuffle = True,
             callbacks = [lr_reducer, csv_logger])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train_arr)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train_arr, y_train_arr, batch_size=batch_size),
                        steps_per_epoch=x_train_arr.shape[0] // batch_size,
                        validation_data=(x_test_arr, y_test_arr),
                        epochs=nb_epoch, verbose=1, max_q_size=100,
                        callbacks=[lr_reducer, csv_logger])

    

Using real-time data augmentation.


/home/ilab/.conda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, validation_data=(array([[[..., steps_per_epoch=12, epochs=200, callbacks=[<keras.ca..., max_queue_size=100)`


Epoch 1/200
12/12 [==============================] - 19s 2s/step - loss: 5.2104 - acc: 0.5299 - val_loss: 5.1774 - val_acc: 0.4510
Epoch 2/200
12/12 [==============================] - 3s 262ms/step - loss: 4.6900 - acc: 0.5229 - val_loss: 4.5529 - val_acc: 0.6225
Epoch 3/200
12/12 [==============================] - 3s 238ms/step - loss: 4.4730 - acc: 0.5917 - val_loss: 4.3772 - val_acc: 0.6225
Epoch 4/200
12/12 [==============================] - 3s 239ms/step - loss: 4.2553 - acc: 0.6344 - val_loss: 4.2153 - val_acc: 0.6422
Epoch 5/200
12/12 [==============================] - 3s 240ms/step - loss: 4.0337 - acc: 0.6918 - val_loss: 5.2084 - val_acc: 0.4657
Epoch 6/200
12/12 [==============================] - 3s 240ms/step - loss: 3.8700 - acc: 0.6330 - val_loss: 3.7903 - val_acc: 0.6225
Epoch 7/200
12/12 [==============================] - 3s 243ms/step - loss: 3.6540 - acc: 0.7044 - val_loss: 3.6681 - val_acc: 0.6127
Epoch 8/200
12/12 [==============================] - 3s 239ms/step - lo

12/12 [==============================] - 3s 239ms/step - loss: 1.2313 - acc: 0.8464 - val_loss: 1.3809 - val_acc: 0.8039
Epoch 63/200
12/12 [==============================] - 3s 242ms/step - loss: 1.2471 - acc: 0.8320 - val_loss: 1.3853 - val_acc: 0.7892
Epoch 64/200
12/12 [==============================] - 3s 238ms/step - loss: 1.2202 - acc: 0.8614 - val_loss: 1.3920 - val_acc: 0.7843
Epoch 65/200
12/12 [==============================] - 3s 238ms/step - loss: 1.2411 - acc: 0.8379 - val_loss: 1.3784 - val_acc: 0.7941
Epoch 66/200
12/12 [==============================] - 3s 242ms/step - loss: 1.2514 - acc: 0.8320 - val_loss: 1.3704 - val_acc: 0.8088
Epoch 67/200
12/12 [==============================] - 3s 236ms/step - loss: 1.2333 - acc: 0.8212 - val_loss: 1.3739 - val_acc: 0.8088
Epoch 68/200
12/12 [==============================] - 3s 239ms/step - loss: 1.2199 - acc: 0.8467 - val_loss: 1.3696 - val_acc: 0.8088
Epoch 69/200
12/12 [==============================] - 3s 241ms/step - loss:

12/12 [==============================] - 3s 243ms/step - loss: 1.2221 - acc: 0.8477 - val_loss: 1.3785 - val_acc: 0.7990
Epoch 124/200
12/12 [==============================] - 3s 241ms/step - loss: 1.2474 - acc: 0.8415 - val_loss: 1.3786 - val_acc: 0.7990
Epoch 125/200
12/12 [==============================] - 3s 240ms/step - loss: 1.2326 - acc: 0.8304 - val_loss: 1.3778 - val_acc: 0.7990
Epoch 126/200
12/12 [==============================] - 3s 240ms/step - loss: 1.2157 - acc: 0.8484 - val_loss: 1.3782 - val_acc: 0.7990
Epoch 127/200
12/12 [==============================] - 3s 239ms/step - loss: 1.2293 - acc: 0.8422 - val_loss: 1.3786 - val_acc: 0.7990
Epoch 128/200
12/12 [==============================] - 3s 239ms/step - loss: 1.2106 - acc: 0.8588 - val_loss: 1.3788 - val_acc: 0.7990
Epoch 129/200
12/12 [==============================] - 3s 240ms/step - loss: 1.2178 - acc: 0.8408 - val_loss: 1.3791 - val_acc: 0.7941
Epoch 130/200
12/12 [==============================] - 3s 238ms/step 

Epoch 184/200
12/12 [==============================] - 3s 240ms/step - loss: 1.2197 - acc: 0.8428 - val_loss: 1.3783 - val_acc: 0.7941
Epoch 185/200
12/12 [==============================] - 3s 239ms/step - loss: 1.2043 - acc: 0.8532 - val_loss: 1.3779 - val_acc: 0.7990
Epoch 186/200
12/12 [==============================] - 3s 243ms/step - loss: 1.2226 - acc: 0.8457 - val_loss: 1.3787 - val_acc: 0.7941
Epoch 187/200
12/12 [==============================] - 3s 239ms/step - loss: 1.2286 - acc: 0.8369 - val_loss: 1.3779 - val_acc: 0.7990
Epoch 188/200
12/12 [==============================] - 3s 239ms/step - loss: 1.2204 - acc: 0.8526 - val_loss: 1.3780 - val_acc: 0.7990
Epoch 189/200
12/12 [==============================] - 3s 239ms/step - loss: 1.2329 - acc: 0.8376 - val_loss: 1.3785 - val_acc: 0.7990
Epoch 190/200
12/12 [==============================] - 3s 241ms/step - loss: 1.2194 - acc: 0.8607 - val_loss: 1.3781 - val_acc: 0.7941
Epoch 191/200
12/12 [==============================] - 